In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射变换：y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去批大小维度的其余维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [44]:
# class Net(nn.Module):

#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1,6,5)
#         self.conv2 = nn.Conv2d(6,16,5)
        
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2=  nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10) 
        
#     def forward(self , x):
#         # 2 * 2 Max pooling
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) 
#         x = F.max_pool2d(F.relu(self.conv2(x)) , 2)
#         x = x.view(-1 ,self.num_flat_features(x)) 
#         x = F.relu(self.fc1(x))
#         x = F.relu(relf.fc2(x))
#         x = self.fc3(x)
#         return x
    
#     def num_flat_features(self , x):
#         size = x.size()[1:] 
#         num_features = 1 
#         for s in size: 
#             num_features *= s
#         return num_features
    
    
# net = Net
# print(net)

In [45]:
params = list(net.parameters())

In [46]:
input = torch.randn(1,1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0432, -0.0609,  0.0995,  0.0124,  0.1545, -0.0774,  0.1199,  0.0669,
         -0.0648,  0.1064]], grad_fn=<AddmmBackward>)


In [47]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [48]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.8839, grad_fn=<MseLossBackward>)


In [49]:
net.zero_grad()     # 清零所有参数（parameter）的梯度缓存

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0045,  0.0192,  0.0142, -0.0268, -0.0160,  0.0110])


In [51]:
import torch.optim as optim

# 创建优化器（optimizer）
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练的迭代中：
optimizer.zero_grad()   # 清零梯度缓存
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 更新参数